<a href="https://colab.research.google.com/github/lazarinastoy/mlfoseo/blob/fuzzy-matching/Fuzzy_Booster_by_Natzir_Ruiz_for_MLforSEO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
import google.auth
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.colab import files
import datetime
from dateutil import relativedelta
import pandas as pd
from collections import defaultdict
from bs4 import BeautifulSoup
import requests
import chardet
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
import unicodedata
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz
!pip install openai
from openai import OpenAI

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


In [ ]:
# Select the date range
end_date = datetime.date.today()
start_date = end_date - relativedelta.relativedelta(months=16)

# Select the site
site = 'https://www.analistaseo.es/' # https://www.example.com/ for a URL-prefix property or sc-domain:example.com for a Domain property

home = "^"+site+"$" # Regex to match the home page

branded_queries = 'natzir|analistaseo|analista seo' # Branded queries to exclude

# Request payload for Google Search Console API
request = {
    'startDate': datetime.datetime.strftime(start_date,'%Y-%m-%d'),
    'endDate': datetime.datetime.strftime(end_date,'%Y-%m-%d'),
    'dimensions': ['page','query'],
    'dimensionFilterGroups': [
    {
      'filters': [
        { # Excluding home page
          'dimension': 'page',
          'operator': 'excludingRegex',
          'expression': home
        },
        { # Excluding branded queries
          'dimension': 'query',
          'operator': 'excludingRegex',
          'expression': branded_queries
        }
      ]
    }
  ],
    'rowLimit': 25000 # Up to 25.000 urls
}

# Upload credentials file
uploaded = files.upload()

# Save the credentials file
with open('credentials.json', 'wb') as f:
    f.write(uploaded[list(uploaded.keys())[0]])

# Set the scope for Google API
SCOPES = ['https://www.googleapis.com/auth/webmasters.readonly']

# Load credentials
creds = None
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        flow.redirect_uri = 'urn:ietf:wg:oauth:2.0:oob'
        auth_url, _ = flow.authorization_url(prompt='consent')

        print(f'Please verify your account at the following URL: {auth_url}')

        # Wait for the verification code
        code = input('Enter the verification code: ')
        flow.fetch_token(code=code)
        creds = flow.credentials

    with open('token.json', 'w') as token:
        token.write(creds.to_json())

# Build the Search Console service
service = build('webmasters', 'v3', credentials=creds)

# Execute the request to the Search Console API
response = service.searchanalytics().query(siteUrl=site, body=request).execute()
print("Getting Google Search Console data...")

# Parse the JSON response
scDict = defaultdict(list)

for row in response['rows']:
    scDict['page'].append(row['keys'][0] or 0)
    scDict['query'].append(row['keys'][1] or 0)
    scDict['clicks'].append(row['clicks'] or 0)
    scDict['ctr'].append(row['ctr'] or 0)
    scDict['impressions'].append(row['impressions'] or 0)
    scDict['position'].append(row['position'] or 0)

# Create a DataFrame from the parsed data
df = pd.DataFrame(data = scDict)

# Convert data types and adjust values
df['clicks'] = df['clicks'].astype('int')
df['ctr'] = df['ctr']*100
df['impressions'] = df['impressions'].astype('int')
df['position'] = df['position'].round(2)
df.sort_values('clicks', inplace = True, ascending = False)

# Get the most clicked query for each page
df = df.drop_duplicates(subset=['page'], keep='first')
df

Saving client_secret_847187862506-4v0hjb96mkgqmb35d9gs6e4cb7l1c1sp.apps.googleusercontent.com.json to client_secret_847187862506-4v0hjb96mkgqmb35d9gs6e4cb7l1c1sp.apps.googleusercontent.com.json
Verifica tu cuenta en el siguiente enlace: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=847187862506-4v0hjb96mkgqmb35d9gs6e4cb7l1c1sp.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fwebmasters.readonly&state=e3V7l8aJf30mvDLDNd8dSNPjBQwv3g&prompt=consent&access_type=offline
Introduce el código de verificación: 4/1AcvDMrCyN6XgaTEahrSs0zvKUSyEN6t5QgreBpKYqvCnrviFuOuHGi5_b3o
Getting Google Search Console...


,page,query,clicks,ctr,impressions,position
0,https://www.analistaseo.es/conversion/modelo-p...,modelo de probabilidad de elaboración,76,7.826982,971,2.74
1,https://www.analistaseo.es/posicionamiento-bus...,buscador semantico,50,7.485030,668,2.46
5,https://www.analistaseo.es/posicionamiento-bus...,navboost,25,4.901961,510,37.94
6,https://www.analistaseo.es/google-api-indexing...,google colab indexing api,21,0.264517,7939,7.90
7,https://www.analistaseo.es/posicionamiento-bus...,cynefin,16,0.410572,3897,9.30
...,...,...,...,...,...,...
1399,https://www.analistaseo.es/conversion/neuromar...,neuromarketing barcelona,0,0.000000,3,77.00
1497,https://www.analistaseo.es/posicionamiento-bus...,análisis seo,0,0.000000,1,9.00
1728,https://www.analistaseo.es/posicionamiento-bus...,logeados,0,0.000000,1,78.00
1773,https://www.analistaseo.es/posicionamiento-bus...,link rel alternate hreflang,0,0.000000,60,64.42


In [ ]:
# Function to extract metadata from a URL
def get_meta(url):
    try:
        response = requests.get(url)
        encoding = chardet.detect(response.content)['encoding']
        if encoding:
            page_content = response.content.decode(encoding)
        else:
            page_content = response.content
        soup = BeautifulSoup(page_content, 'html.parser')
        title = soup.find('title').get_text() if soup.find('title') else 'No title' # Get the title
        meta = soup.select('meta[name="description"]')[0].attrs["content"] if soup.select('meta[name="description"]') else 'No meta description' # Get the meta description
        h1 = soup.find('h1').get_text() if soup.find('h1') else 'No h1' # Get the first h1
        return title, meta, h1
    except Exception as e:
        return 'Error', 'Error', 'Error'

# Apply the function and add the results to the DataFrame
df['title'], df['meta'], df['h1'] = zip(*df['page'].apply(get_meta))
df

,page,query,clicks,ctr,impressions,position,title,meta,h1
0,https://www.analistaseo.es/conversion/modelo-p...,modelo de probabilidad de elaboración,76,7.826982,971,2.74,Modelo de Probabilidad de Elaboración y Persua...,"El Modelo de la Probabilidad de Elaboración, t...",Cómo usar el Modelo de la Probabilidad de Elab...
1,https://www.analistaseo.es/posicionamiento-bus...,buscador semantico,50,7.485030,668,2.46,Qué es un buscador semántico,Un buscador semántico es aquel que no da enlac...,Qué es un buscador semántico
5,https://www.analistaseo.es/posicionamiento-bus...,navboost,25,4.901961,510,37.94,Los algoritmos de Google al descubierto. Cómo ...,\nEn este artículo nos adentramos en el funcio...,Los algoritmos de Google al descubierto. Cómo ...
6,https://www.analistaseo.es/google-api-indexing...,google colab indexing api,21,0.264517,7939,7.90,API Indexing Test with Google Colab,No meta description,API Indexing Test
7,https://www.analistaseo.es/posicionamiento-bus...,cynefin,16,0.410572,3897,9.30,Aplicando el marco de Cynefin en la toma de de...,\nTomar decisiones acertadas en el volátil mun...,Aplicando el marco de Cynefin en la toma de de...
...,...,...,...,...,...,...,...,...,...
1399,https://www.analistaseo.es/conversion/neuromar...,neuromarketing barcelona,0,0.000000,3,77.00,Neuromarketing en eShow Barcelona 2014,"El 95% de nuestros pensamientos, emociones y a...",Neuromarketing en eShow Barcelona 2014
1497,https://www.analistaseo.es/posicionamiento-bus...,análisis seo,0,0.000000,1,9.00,Artículos de Posicionamiento en Buscadores (SEO),No meta description,Artículos de SEO
1728,https://www.analistaseo.es/posicionamiento-bus...,logeados,0,0.000000,1,78.00,Búsqueda Segura en Google (SSL Search),Desde ayer todos los usuarios que se identifiq...,Búsqueda Segura en Google (SSL Search)
1773,https://www.analistaseo.es/posicionamiento-bus...,link rel alternate hreflang,0,0.000000,60,64.42,Caso de éxito SEO Internacional con HrefLang,El caso de éxito SEO que os enseño a continuac...,Caso de éxito SEO Internacional con HrefLang


In [ ]:
lang = 'spanish'  # Language for stopwords

# Function to clean text
def clean_text(text):
    text = str(text).lower().strip()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = unicodedata.normalize('NFKD', text).encode('ascii', errors='ignore').decode('utf-8')

    stop_words = set(stopwords.words(lang))
    words = text.split()
    words_cleaned = [word for word in words if word.lower() not in stop_words]
    return ' '.join(words_cleaned)

# Clean specified columns
columns_to_clean = ['title', 'meta', 'h1', 'query']

for column in columns_to_clean:
    df[column + '_clean'] = df[column].apply(clean_text)

df

,page,query,clicks,ctr,impressions,position,title,meta,h1,title_clean,meta_clean,h1_clean,query_clean
0,https://www.analistaseo.es/conversion/modelo-p...,modelo de probabilidad de elaboración,76,7.826982,971,2.74,Modelo de Probabilidad de Elaboración y Persua...,"El Modelo de la Probabilidad de Elaboración, t...",Cómo usar el Modelo de la Probabilidad de Elab...,modelo probabilidad elaboracion persuasion web,modelo probabilidad elaboracion tambien conoci...,usar modelo probabilidad elaboracion persuadir...,modelo probabilidad elaboracion
1,https://www.analistaseo.es/posicionamiento-bus...,buscador semantico,50,7.485030,668,2.46,Qué es un buscador semántico,Un buscador semántico es aquel que no da enlac...,Qué es un buscador semántico,buscador semantico,buscador semantico aquel da enlaces clasificad...,buscador semantico,buscador semantico
5,https://www.analistaseo.es/posicionamiento-bus...,navboost,25,4.901961,510,37.94,Los algoritmos de Google al descubierto. Cómo ...,\nEn este artículo nos adentramos en el funcio...,Los algoritmos de Google al descubierto. Cómo ...,algoritmos google descubierto funciona buscado...,articulo adentramos funcionamiento interno goo...,algoritmos google descubierto funciona buscado...,navboost
6,https://www.analistaseo.es/google-api-indexing...,google colab indexing api,21,0.264517,7939,7.90,API Indexing Test with Google Colab,No meta description,API Indexing Test,api indexing test with google colab,meta description,api indexing test,google colab indexing api
7,https://www.analistaseo.es/posicionamiento-bus...,cynefin,16,0.410572,3897,9.30,Aplicando el marco de Cynefin en la toma de de...,\nTomar decisiones acertadas en el volátil mun...,Aplicando el marco de Cynefin en la toma de de...,aplicando marco cynefin toma decisiones gestio...,tomar decisiones acertadas volatil mundo seo p...,aplicando marco cynefin toma decisiones gestio...,cynefin
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399,https://www.analistaseo.es/conversion/neuromar...,neuromarketing barcelona,0,0.000000,3,77.00,Neuromarketing en eShow Barcelona 2014,"El 95% de nuestros pensamientos, emociones y a...",Neuromarketing en eShow Barcelona 2014,neuromarketing eshow barcelona,pensamientos emociones aprendizajes ocurren fo...,neuromarketing eshow barcelona,neuromarketing barcelona
1497,https://www.analistaseo.es/posicionamiento-bus...,análisis seo,0,0.000000,1,9.00,Artículos de Posicionamiento en Buscadores (SEO),No meta description,Artículos de SEO,articulos posicionamiento buscadores seo,meta description,articulos seo,analisis seo
1728,https://www.analistaseo.es/posicionamiento-bus...,logeados,0,0.000000,1,78.00,Búsqueda Segura en Google (SSL Search),Desde ayer todos los usuarios que se identifiq...,Búsqueda Segura en Google (SSL Search),busqueda segura google ssl search,ayer usuarios identifiquen version com google ...,busqueda segura google ssl search,logeados
1773,https://www.analistaseo.es/posicionamiento-bus...,link rel alternate hreflang,0,0.000000,60,64.42,Caso de éxito SEO Internacional con HrefLang,El caso de éxito SEO que os enseño a continuac...,Caso de éxito SEO Internacional con HrefLang,caso exito seo internacional hreflang,caso exito seo enseno continuacion muestra imp...,caso exito seo internacional hreflang,link rel alternate hreflang


In [ ]:
# Fuzzy matching to compute similarity with token_set_ratio https://pypi.org/project/fuzzywuzzy/

columns = ['title_clean', 'meta_clean', 'h1_clean']

for col in columns:
    similarity = []
    for index, row in df.iterrows():
        sim = fuzz.token_set_ratio(row['query_clean'], row[col])
        similarity.append(sim)
    df[f'{col}_similarity'] = similarity

# Rename columns for clarity
df.rename(columns=lambda x: x.replace('_clean_similarity', '_similarity') if x.endswith('_clean_similarity') else x, inplace=True)
columns_to_drop = [col for col in df.columns if '_clean' in col]
df.drop(columns=columns_to_drop, inplace=True)

df

,page,query,clicks,ctr,impressions,position,title,meta,h1,title_similarity,meta_similarity,h1_similarity
0,https://www.analistaseo.es/conversion/modelo-p...,modelo de probabilidad de elaboración,76,7.826982,971,2.74,Modelo de Probabilidad de Elaboración y Persua...,"El Modelo de la Probabilidad de Elaboración, t...",Cómo usar el Modelo de la Probabilidad de Elab...,100,100,100
1,https://www.analistaseo.es/posicionamiento-bus...,buscador semantico,50,7.485030,668,2.46,Qué es un buscador semántico,Un buscador semántico es aquel que no da enlac...,Qué es un buscador semántico,100,100,100
5,https://www.analistaseo.es/posicionamiento-bus...,navboost,25,4.901961,510,37.94,Los algoritmos de Google al descubierto. Cómo ...,\nEn este artículo nos adentramos en el funcio...,Los algoritmos de Google al descubierto. Cómo ...,12,11,12
6,https://www.analistaseo.es/google-api-indexing...,google colab indexing api,21,0.264517,7939,7.90,API Indexing Test with Google Colab,No meta description,API Indexing Test,100,20,83
7,https://www.analistaseo.es/posicionamiento-bus...,cynefin,16,0.410572,3897,9.30,Aplicando el marco de Cynefin en la toma de de...,\nTomar decisiones acertadas en el volátil mun...,Aplicando el marco de Cynefin en la toma de de...,100,9,100
...,...,...,...,...,...,...,...,...,...,...,...,...
1399,https://www.analistaseo.es/conversion/neuromar...,neuromarketing barcelona,0,0.000000,3,77.00,Neuromarketing en eShow Barcelona 2014,"El 95% de nuestros pensamientos, emociones y a...",Neuromarketing en eShow Barcelona 2014,100,18,100
1497,https://www.analistaseo.es/posicionamiento-bus...,análisis seo,0,0.000000,1,9.00,Artículos de Posicionamiento en Buscadores (SEO),No meta description,Artículos de SEO,40,7,56
1728,https://www.analistaseo.es/posicionamiento-bus...,logeados,0,0.000000,1,78.00,Búsqueda Segura en Google (SSL Search),Desde ayer todos los usuarios que se identifiq...,Búsqueda Segura en Google (SSL Search),24,10,24
1773,https://www.analistaseo.es/posicionamiento-bus...,link rel alternate hreflang,0,0.000000,60,64.42,Caso de éxito SEO Internacional con HrefLang,El caso de éxito SEO que os enseño a continuac...,Caso de éxito SEO Internacional con HrefLang,50,46,50


Optional step: Generate new titles if the similarity between the current title and the top-performing query is less than 60. Feel free to adjust the similarity threshold based on the results you observe and generate new descriptions and h1 tags. This step may incur a significant cost depending on the size of your dataset.

In [ ]:
# Optional step to generate new titles if similarity is below a threshold
filtered_df = df[df['title_similarity'] <= 60]

# Configure OpenAI API
client = OpenAI(api_key="YOUR OPENAI TOKEN HERE")

MODEL = "gpt-4o"
TEMPERATURES = [0.0]

PROMPT_TEMPLATE = """Task: Generate a new title that meets the following criteria:
1. The title cannot be longer than 60 characters.
2. The title must include the specified query, represented in the template as '{query}'.
3. The title must be written in the specified language, represented in the template as '{lang}'.
4. No words should be repeated in the title.
5. Use the entities found in the given 'current title' '{title}' to construct the new title.
Please ensure that your title meets all of the criteria.
"""

# Function to generate a new title using OpenAI
def generar_titulo(prompt, temperature):
    completion = client.chat.completions.create(
        model=MODEL,
        temperature=temperature,
        messages=[{"role": "user", "content": prompt}],
    )
    return completion.choices[0].message.content.strip()

# Generate new titles for each filtered row
for index, row in filtered_df.iterrows():
    query = row['query']
    current_title = row['title']
    prompt = PROMPT_TEMPLATE.format(query=query, lang=lang, title=current_title)
    new_title = generar_titulo(prompt, TEMPERATURES[0])
    df.at[index, 'new_title'] = new_title

# Function to strip quotes from the beginning and end of a string
def strip_quotes(title):
    if title.startswith('"') and title.endswith('"'):
        return title[1:-1]
    return title

# Apply the function to the 'new_title' column
df['new_title'] = df['new_title'].apply(strip_quotes)

# Save the results to a CSV file
df.to_csv('fuzzy_booster.csv', index=False)

# Download the CSV file
files.download('fuzzy_booster.csv')

df

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,page,query,clicks,ctr,impressions,position,title,meta,h1,title_similarity,meta_similarity,h1_similarity,new_title
0,https://www.analistaseo.es/conversion/modelo-p...,modelo de probabilidad de elaboración,76,7.826982,971,2.74,Modelo de Probabilidad de Elaboración y Persua...,"El Modelo de la Probabilidad de Elaboración, t...",Cómo usar el Modelo de la Probabilidad de Elab...,100,100,100,nan
1,https://www.analistaseo.es/posicionamiento-bus...,buscador semantico,50,7.485030,668,2.46,Qué es un buscador semántico,Un buscador semántico es aquel que no da enlac...,Qué es un buscador semántico,100,100,100,nan
5,https://www.analistaseo.es/posicionamiento-bus...,navboost,25,4.901961,510,37.94,Los algoritmos de Google al descubierto. Cómo ...,\nEn este artículo nos adentramos en el funcio...,Los algoritmos de Google al descubierto. Cómo ...,12,11,12,Navboost: Algoritmos de Google y Documentos Fi...
6,https://www.analistaseo.es/google-api-indexing...,google colab indexing api,21,0.264517,7939,7.90,API Indexing Test with Google Colab,No meta description,API Indexing Test,100,20,83,nan
7,https://www.analistaseo.es/posicionamiento-bus...,cynefin,16,0.410572,3897,9.30,Aplicando el marco de Cynefin en la toma de de...,\nTomar decisiones acertadas en el volátil mun...,Aplicando el marco de Cynefin en la toma de de...,100,9,100,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399,https://www.analistaseo.es/conversion/neuromar...,neuromarketing barcelona,0,0.000000,3,77.00,Neuromarketing en eShow Barcelona 2014,"El 95% de nuestros pensamientos, emociones y a...",Neuromarketing en eShow Barcelona 2014,100,18,100,nan
1497,https://www.analistaseo.es/posicionamiento-bus...,análisis seo,0,0.000000,1,9.00,Artículos de Posicionamiento en Buscadores (SEO),No meta description,Artículos de SEO,40,7,56,Análisis SEO: Artículos de Posicionamiento en ...
1728,https://www.analistaseo.es/posicionamiento-bus...,logeados,0,0.000000,1,78.00,Búsqueda Segura en Google (SSL Search),Desde ayer todos los usuarios que se identifiq...,Búsqueda Segura en Google (SSL Search),24,10,24,Búsqueda Segura de Google para Usuarios Logeados
1773,https://www.analistaseo.es/posicionamiento-bus...,link rel alternate hreflang,0,0.000000,60,64.42,Caso de éxito SEO Internacional con HrefLang,El caso de éxito SEO que os enseño a continuac...,Caso de éxito SEO Internacional con HrefLang,50,46,50,Éxito Internacional con link rel alternate hre...
